# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [21]:
import pandas as pd
data=pd.read_csv('/datasets/data.csv')
data.head()
#data.info()


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


**Вывод**

## Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:
data['total_income'] = data['total_income'].fillna(0)#меняем все пропущенные значение на 0
data['days_employed'] = data['days_employed'].fillna(0)
data['days_employed'] = pd.to_numeric(data['days_employed']) #меняем тип данных, чтобы можно было проводить мат. операции
data['days_employed'] = data['days_employed'].abs()#делаем значения по модулю
data.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [3]:
data['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

In [4]:
data.groupby('income_type')['days_employed'].transform('median')
data['work_days'] = data.groupby('income_type')['days_employed'].transform('median')
data.groupby('income_type')['total_income'].transform('median')
data['dohod'] = data.groupby('income_type')['total_income'].transform('median')
data.loc[data['days_employed'] == 0, 'days_employed'] = data['work_days']
data.loc[data['total_income'] == 0, 'total_income'] = data['dohod']
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,work_days,dohod
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,1360.363902,133546.457238
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,1360.363902,133546.457238
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,1360.363902,133546.457238
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,1360.363902,133546.457238
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,360505.668544,110179.690761


In [5]:
data['children'] = data['children'].abs()
data.loc[data['children'] ==20, 'children'] = 2

In [6]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,work_days,dohod
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,1360.363902,133546.457238
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,1360.363902,133546.457238
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,1360.363902,133546.457238
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,1360.363902,133546.457238
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,360505.668544,110179.690761


**Вывод**

-Было принято решение заменить все пропуски в стобцах days_employed и ежемесячный доход, исходя из того, что ноль отработанных дней равно нулевому доходу
-В таблице отображались некорректные значение детей(-1;20) исходя из логики, что не божет быть меньше одного ребенка или двадцать детей, было принято решение переметить их в категорию (1;20)


Возможноая причина пропусков может быть, потому что клиент работает на не официальной работе

### Замена типа данных

In [7]:
data['days_employed']=data['days_employed'].astype('int')
data['total_income']=data['total_income'].astype('int')
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,work_days,dohod
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,1360.363902,133546.457238
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,1360.363902,133546.457238
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,1360.363902,133546.457238
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,1360.363902,133546.457238
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,360505.668544,110179.690761


**Вывод**

Замена типа данный float а integer, т.к. удобнее смотреть на столбцы days_employed и total_income, когда там целые числа

### Обработка дубликатов

In [8]:
data.duplicated('education').sum()#подсчет дубликатов

21510

In [9]:
data['education'].value_counts()#вывод уникальных значений

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

In [10]:
data['education_lowercase'] = data['education'].str.lower()#сводим значения к одному регистру
data['education_lowercase'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education_lowercase, dtype: int64

In [22]:

#data=data.dropna().reset_index(drop = True)#удаляем пропущенные значение с сохранением регистра
data = data.drop_duplicates().reset_index(drop=True) 
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


**Вывод**

Был выбран метод duplicated().sum() для того, чтобы посчитать количество дубликатов в наборе данных

Появление дубликатов может быть вызвано ошибками ввода данных, например одно и тоже несколько раз

### Лемматизация

In [23]:
from pymystem3 import Mystem
m = Mystem()
def lemmed_purpose(row):
    
    return m.lemmatize(row['purpose'])      
data['lemmed_purpose']= data.apply(lemmed_purpose, axis=1)
data.head()


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmed_purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,"[покупка, , жилье, \n]"
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,"[приобретение, , автомобиль, \n]"
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,"[покупка, , жилье, \n]"
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,"[дополнительный, , образование, \n]"
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,"[сыграть, , свадьба, \n]"


**Вывод**

Была сделанна лемматизация по строкам purpose, чтобы было удобнее, например, в дальнейшем осуществлять поиск по ключевым словам 

### Категоризация данных

In [14]:
data_log=data[['education_lowercase', 'education_id']]#группируем таблицу по двум стобцам
data_log=data_log.drop_duplicates().reset_index(drop = True)
data_log


,education_lowercase,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


In [15]:
data_log1=data[['family_status', 'family_status_id']]
data_log1=data_log1.drop_duplicates().reset_index(drop = True)
data_log1

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


**Вывод**

Были выбраны словари по стобцам education_lowercase и family_status, чтобы можно было наперед предсказать финансовые возможности и остаток свободных средств клиента и их индефикационным номерам, для удобного поиска


## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [16]:
data_pivot2 = data.pivot_table(index=['children'], values='debt', aggfunc='sum')#число тех кто имеет долг по кредиту
data_pivot3 = data.pivot_table(index=['children'], values='debt', aggfunc='count')#общ кол-во тех кто подходит под категорию
data_children = data_pivot2.merge(data_pivot3, on='children', how='right')
data_children['debt_ratio']=(data_children['debt_x']/data_children['debt_y'])*100
print('Процент задолженностей по кредиту в зависимости от числа детей:', data_children['debt_ratio'])  

Процент задолженностей по кредиту в зависимости от числа детей: children
0    7.535266
1    9.163921
2    9.492481
3    8.181818
4    9.756098
5    0.000000
Name: debt_ratio, dtype: float64


**Вывод**

Исходя из полученных данных, можно сказать, что клиенты без детей более платежеспособны, чем клиенты с детьми, на которых уходит значительная часть средтсв

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [17]:
data_pivot2 = data.pivot_table(index=['family_status'], values='debt', aggfunc='sum')#число тех кто имеет долг по кредиту
data_pivot3 = data.pivot_table(index=['family_status'], values='debt', aggfunc='count')#общ кол-во тех кто подходит под категорию
data_family_status = data_pivot2.merge(data_pivot3, on='family_status', how='right')
data_family_status['debt_ratio']=(data_family_status['debt_x']/data_family_status['debt_y'])*100
print('Процент задолженностей по кредиту в зависимости от семейного положения:', data_family_status['debt_ratio'])

Процент задолженностей по кредиту в зависимости от семейного положения: family_status
Не женат / не замужем    9.750890
в разводе                7.112971
вдовец / вдова           6.569343
гражданский брак         9.320202
женат / замужем          7.542126
Name: debt_ratio, dtype: float64


**Вывод**

Исходя из полученных данных, можно сказать, что клиенты, входящие в категорию "вдовец / вдова", лучше выплачивают кредит 

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [18]:
avg = data['total_income'].median()
def zavisimost3(row):
    total_income=row['total_income']
    if total_income >= avg:
        if total_income < avg*2:
            return 'Уровень дохода средний'
    if total_income < avg:
        if total_income > avg/2:
            return 'Уровень дохода ниже среднего'
    if total_income >= avg*2:
        return 'Уровень дохода выше среднего'
    
data['importance_status3'] = data.apply(zavisimost3, axis=1) 
data['importance_status3'].value_counts()
data_pivot2 = data.pivot_table(index=['importance_status3'], values='debt', aggfunc='sum')#число тех кто имеет долг по кредиту
data_pivot3 = data.pivot_table(index=['importance_status3'], values='debt', aggfunc='count')#общ кол-во тех кто подходит под категорию
data_importance_status3 = data_pivot2.merge(data_pivot3, on='importance_status3', how='right')
data_importance_status3['debt_ratio']=(data_importance_status3['debt_x']/data_importance_status3['debt_y'])*100
print('Процент задолженностей по кредиту в зависимости от уровня дохода: ', data_importance_status3['debt_ratio'])        

Процент задолженностей по кредиту в зависимости от уровня дохода:  importance_status3
Уровень дохода выше среднего    7.097458
Уровень дохода ниже среднего    8.498002
Уровень дохода средний          8.126130
Name: debt_ratio, dtype: float64


In [19]:
data['importance_status3'].value_counts()

Уровень дохода ниже среднего    9261
Уровень дохода средний          8848
Уровень дохода выше среднего    1888
Name: importance_status3, dtype: int64

**Вывод**

Отчевидно, что клиенты с достатком выше среднего имеют самую низкую задолженность. У остальных же процент долгов практически одинаковый, возможно это связано с тем, что их количесво близко к друг другу и их стоит разбить на более мелкие категрии.

- Как разные цели кредита влияют на его возврат в срок?

In [20]:
data_pivot2 = data.pivot_table(index=['purpose'], values='debt', aggfunc='sum')#число тех кто имеет долг по кредиту
data_pivot3 = data.pivot_table(index=['purpose'], values='debt', aggfunc='count')#общ кол-во тех кто подходит под категорию
data_purpose = data_pivot2.merge(data_pivot3, on='purpose', how='right')
data_purpose['debt_ratio']=(data_purpose['debt_x']/data_purpose['debt_y'])*100
print('Процент задолженностей по кредиту в зависимости от его цели: ', data_purpose['debt_ratio'])  

Процент задолженностей по кредиту в зависимости от его цели:  purpose
автомобили                                 9.205021
автомобиль                                 8.502024
высшее образование                         8.849558
дополнительное образование                 8.260870
жилье                                      7.120743
заняться высшим образованием               8.669355
заняться образованием                      9.558824
на покупку автомобиля                      9.322034
на покупку подержанного автомобиля         7.531381
на покупку своего автомобиля               9.108911
на проведение свадьбы                      8.279431
недвижимость                               6.635071
образование                                7.158837
операции с жильем                          7.361963
операции с коммерческой недвижимостью      8.000000
операции с недвижимостью                   8.148148
операции со своей недвижимостью            7.974482
покупка жилой недвижимости                 6.7

**Вывод**

Исходя из полученных данных, можно сказать, что клиенты, имеют большой процент задолжности в категории образование, а это как правило студенты, которые не имеют большого дохода, и сделки с автомобилем.

## Шаг 4. Общий вывод

Вывод по всему можно сделать такой, что хуже всего выплачивают кредит клиенты, которые находятся в гражнском браке с детьми, у которых доход ниже среднего и хотят свой автомобиль.

Также можно заметить артефакт при подсчете процента задолженностей по кредиту в зависимости от числа детей. У клиентов, которых пятеро детей выдает 0%, а такое невозможно.